In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

CSV_COLUMN_NAMES = ['SpealLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

#Load Dataset:
train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0) #header=0 betyder bare at row 0 er header
test = pd.read_csv(test_path,names=CSV_COLUMN_NAMES, header=0)

train_y = train.pop('Species')
test_y = test.pop('Species')
print(train.head())

#Input Funciton:
def input_fn(features, labels, training=True, batch_size=256):
    #Konveterer data til dataset:
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    #Shuffle hvis det er træningsdata:
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

#Feature Columns:
my_feature_columns =[]
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

#Building The Model:
#tf.get_logger().setLevel('INFO') #Giver output mens den træner DNN.

classifier = tf.estimator.DNNClassifier(feature_columns=my_feature_columns,hidden_units=[30,10], n_classes=3)

classifier.train(input_fn=lambda: input_fn(train, train_y, training=True),steps=5000)

   SpealLength  SepalWidth  PetalLength  PetalWidth
0          6.4         2.8          5.6         2.2
1          5.0         2.3          3.3         1.0
2          4.9         2.5          4.5         1.7
3          4.9         3.1          1.5         0.1
4          5.7         3.8          1.7         0.3
[NumericColumn(key='SpealLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\LUCASB~1\\AppData\\Local\\Temp\\tmpkyyrthdq', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_confi

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\LUCASB~1\AppData\Local\Temp\tmpkyyrthdq\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 3.1741543, step = 0
INFO:tensorflow:global_step/sec: 605.51
INFO:tensorflow:loss = 2.0569966, step = 100 (0.167 sec)
INFO:tensorflow:global_step/sec: 786.691
INFO:tensorflow:loss = 1.8319447, step = 200 (0.127 sec)
INFO:tensorflow:global_step/sec: 839.574
INFO:tensorflow:loss = 1.5240601, step = 300 (0.118 sec)
INFO:tensorflow:global_step/sec: 825.694
INFO:tensorflow:loss = 1.3413292, step = 400 (0.122 sec)
INFO:tensorflow:global_step/sec: 839.568
INFO:tensorflow:loss = 1.2394681

In [5]:
#Evaluate Estimator:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))
print('\nTest set Accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2024-03-22T17:04:48
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\LUCASB~1\AppData\Local\Temp\tmpkyyrthdq\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.14713s
INFO:tensorflow:Finished evaluation at 2024-03-22-17:04:48
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.43333334, average_loss = 0.89488536, global_step = 5000, loss = 0.89488536
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\LUCASB~1\AppData\Local\Temp\tmpkyyrthdq\model.ckpt-5000

Test set Accuracy: 0.433



Vi prøver nu at predicte en blomst:

In [6]:
def input_fn(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLegth', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted")
for feature in features:
    valid = True
    while valid:
        val = input(feature + ': ')
        if not val.isdigit(): valid = False

    predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    print(pred_dict)
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100 * probability))

Please type numeric values as prompted
INFO:tensorflow:Calling model_fn.


ValueError: in user code:

    c:\Users\Lucas Bonde\anaconda3\envs\MLFUN\lib\site-packages\tensorflow_estimator\python\estimator\canned\dnn.py:346 call  *
        net = self._input_layer(features, training=is_training)
    c:\Users\Lucas Bonde\anaconda3\envs\MLFUN\lib\site-packages\keras\engine\base_layer_v1.py:765 __call__  **
        outputs = call_fn(cast_inputs, *args, **kwargs)
    c:\Users\Lucas Bonde\anaconda3\envs\MLFUN\lib\site-packages\keras\feature_column\dense_features.py:169 call  **
        self._state_manager)
    c:\Users\Lucas Bonde\anaconda3\envs\MLFUN\lib\site-packages\tensorflow\python\feature_column\feature_column_v2.py:2697 get_dense_tensor
        return transformation_cache.get(self, state_manager)
    c:\Users\Lucas Bonde\anaconda3\envs\MLFUN\lib\site-packages\tensorflow\python\feature_column\feature_column_v2.py:2460 get
        transformed = column.transform_feature(self, state_manager)
    c:\Users\Lucas Bonde\anaconda3\envs\MLFUN\lib\site-packages\tensorflo